# 实验5：卷积神经网络CNN(ResNET)

In [4]:
# 加载 CIFAR-10 数据集
(cifar_train_data, cifar_train_labels), (cifar_test_data, cifar_test_labels) = datasets.cifar10.load_data()
cifar_train_data, cifar_test_data = cifar_train_data / 255.0, cifar_test_data / 255.0
print("train_data: ",cifar_train_data.shape)
print("test_data: ",cifar_test_data.shape)

train_data:  (50000, 32, 32, 3)
test_data:  (10000, 32, 32, 3)


## 模型构建

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, datasets


In [11]:
# resNET-18
class resnet_model(Model):
    def __init__(self, num_classes):
        super(resnet_model, self).__init__()
        # 第一层，卷积
        self.conv1 = layers.Conv2D(64,(3,3),padding='same',activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPooling2D((2,2))
        # 第二层开始，用“块” 
        self.block1 = self.build_resnet_block(64,2)
        self.block2 = self.build_resnet_block(128,2,stride=2)
        self.block3 = self.build_resnet_block(256,2,stride=2)
        self.block4 = self.build_resnet_block(512,2,stride=2)
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes, activation='softmax')
    
    def build_resnet_block(self, filters, num_blocks, stride=1):
        res_block = tf.keras.Sequential()
        res_block.add(BasicBlock(filters, stride))
        for _ in range(1, num_blocks):
            res_block.add(BasicBlock(filters, stride))
            
        return res_block
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        
        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
class BasicBlock(layers.Layer):
    def __init__(self, filters, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(filters, (3,3), strides=stride, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, (3,3), strides=1, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        if stride !=1:
            self.downsample = tf.keras.Sequential([
                layers.Conv2D(filters, (1,1), strides=stride),
                layers.BatchNormalization()
            ])
        else:
            self.downsample = lambda x: x
    
    def call(self, inputs):
        residual = self.downsample(inputs)
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        
        x += residual
        x = tf.nn.relu(x)
        return x
    
        

In [15]:
# GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [18]:
model = resnet_model(num_classes=10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(cifar_train_data, cifar_train_labels, epochs=10, batch_size=64, validation_data=(cifar_test_data,cifar_test_labels))
test_loss, test_acc = model.evaluate(cifar_train_data, cifar_train_labels)

Epoch 1/10
782/782 [==============================] - 808s 1s/step - loss: 1.6714 - accuracy: 0.4127 - val_loss: 1.4371 - val_accuracy: 0.4957
Epoch 2/10
782/782 [==============================] - 803s 1s/step - loss: 1.2084 - accuracy: 0.5755 - val_loss: 1.1583 - val_accuracy: 0.5921
Epoch 3/10
782/782 [==============================] - 803s 1s/step - loss: 0.9925 - accuracy: 0.6546 - val_loss: 1.2060 - val_accuracy: 0.6011
Epoch 4/10
782/782 [==============================] - 798s 1s/step - loss: 0.8301 - accuracy: 0.7136 - val_loss: 0.9276 - val_accuracy: 0.6803
Epoch 5/10
782/782 [==============================] - 803s 1s/step - loss: 0.7211 - accuracy: 0.7538 - val_loss: 0.9282 - val_accuracy: 0.6841
Epoch 6/10
782/782 [==============================] - 802s 1s/step - loss: 0.6207 - accuracy: 0.7878 - val_loss: 0.8793 - val_accuracy: 0.6997
Epoch 7/10
782/782 [==============================] - 804s 1s/step - loss: 0.5413 - accuracy: 0.8147 - val_loss: 0.9208 - val_accuracy: 0.7017

In [19]:
test_acc

0.9160199761390686